In [ ]:
# this loads various objects into the context, see 
# https://docs.kedro.org/en/stable/notebooks_and_ipython/kedro_and_notebooks.html#kedro-line-magics
%load_ext kedro.ipython
%reload_kedro  --env cloud
cat = catalog #just a convenience